In [28]:
import cv2
import numpy as np
from skimage.feature import hog
import os
import time
import pickle


In [29]:
def resizeImage(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.resize(gray , (128 , 64))

In [30]:
def extract_hand_segment(image):
     # Convert BGR image to YCrCb color space
    ycrcb_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # Split the channels
    y_channel, cr_channel, cb_channel = cv2.split(ycrcb_image)

    # Thresholding to create a binary mask for hand region
    _, cr_mask = cv2.threshold(cr_channel, 133, 173, cv2.THRESH_BINARY)
    _, cb_mask = cv2.threshold(cb_channel, 77, 127, cv2.THRESH_BINARY)

    # Combine the masks
    mask = cv2.bitwise_and(cr_mask, cb_mask)
    # Find the contours of the object in the mask
    _, contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour (which should be the hand)
    contour_sizes = [(cv2.contourArea(contour), contour) for contour in contours]
    largest_contour = max(contour_sizes, key=lambda x: x[0])[1]

    # Create a mask of the hand contour
    hand_mask = np.zeros_like(mask)
    cv2.drawContours(hand_mask, [largest_contour], 0, 255, cv2.FILLED)

    # Dilate the hand mask to fill in any gaps
    kernel = np.ones((1, 1), np.uint8)
    dilated_mask = cv2.dilate(hand_mask, kernel, iterations=1)

    # Apply erosion to reduce the size of the segmented area
    kernel = np.ones((1, 1), np.uint8)
    eroded_mask = cv2.erode(dilated_mask, kernel, iterations=1)

    # Apply the hand mask to the original image to extract the hand
    hand_segment = cv2.bitwise_and(image, image, mask=eroded_mask)

    # Return the segmented hand image
    return hand_segment


In [31]:
# Get the current working directory
current_path = os.getcwd()

# Folder name containing the images
folder_name = 'data'

# Construct the folder path
folder_path = os.path.join(current_path, folder_name)

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

print(len(file_list))

60


In [32]:
filename = 'model.sav'
SVM_Modle = pickle.load(open(filename, 'rb'))
lables = []
times = []

for i in range(1, len(file_list) + 1):
    filename = 'data/{}.JPG'.format(i)
    img = cv2.imread(filename)
    if  img is None:
        continue   
   
    startTime = time.time()

    hand_roi = extract_hand_segment(img)
    finalImg = resizeImage(hand_roi)
    hog_features = hog(finalImg, orientations=12, pixels_per_cell=(12, 12), cells_per_block=(3, 3), block_norm='L2-Hys')

    # Reshape hog_features to match SVM model input
    hog_features = hog_features.reshape(1, -1)

    # Make predictions on the testing set
    y_pred = SVM_Modle.predict(hog_features)

    totalTime = time.time() - startTime

    lables.append(y_pred)
    times.append(totalTime)
        


        

In [33]:
# Open a text file in write mode
file_path = 'results.txt'
with open(file_path, 'w') as file:
    for lable in lables:    
        file.write(str(lable[0]) + "\n")
print('Content written to', file_path)

Content written to results.txt


In [34]:
# Open a text file in write mode
file_path = 'time.txt'
with open(file_path, 'w') as file:
    for time in times:    
        file.write(str(round(time , 3)) + '\n')
print('Content written to', file_path)

Content written to time.txt
